In [90]:
import copy
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import pickle
import lightgbm as lgb
import category_encoders as ce

In [91]:
df_bb = pd.read_csv('../crawler/data_bonbanh.csv')
df_oto = pd.read_csv('../crawler/data_oto.csv')

# First clean data for bonbanh


In [92]:
def describle_data(df):
    print('Shape of data:\n', df.shape)
    print('--'*20)
    print('Columns of data: \n', df.columns)
    print('--'*20)
    print('Data types of data: \n', df.dtypes)
    print('--'*20)
    print('Missing values of data: \n', df.isna().sum())
    print('--'*20)
    print('Describe data: \n', df.describe())
    print('--'*20)
    print('Info data: \n', df.info())
describle_data(df_bb)

Shape of data:
 (14212, 9)
----------------------------------------
Columns of data: 
 Index(['car_name', 'year', 'price', 'assemble_place', 'series', 'km',
       'engine_type', 'transmission', 'url'],
      dtype='object')
----------------------------------------
Data types of data: 
 car_name          object
year              object
price             object
assemble_place    object
series            object
km                object
engine_type       object
transmission      object
url               object
dtype: object
----------------------------------------
Missing values of data: 
 car_name          1
year              0
price             0
assemble_place    0
series            0
km                0
engine_type       0
transmission      0
url               0
dtype: int64
----------------------------------------
Describe data: 
                             car_name   year      price      assemble_place  \
count                          14211  14212      14212               14212   

# Xóa xe trùng lặp(do sai sót trong việc cào)

In [93]:
df_bb.drop_duplicates(subset=['url'], inplace=True)
describle_data(df_bb)

Shape of data:
 (14188, 9)
----------------------------------------
Columns of data: 
 Index(['car_name', 'year', 'price', 'assemble_place', 'series', 'km',
       'engine_type', 'transmission', 'url'],
      dtype='object')
----------------------------------------
Data types of data: 
 car_name          object
year              object
price             object
assemble_place    object
series            object
km                object
engine_type       object
transmission      object
url               object
dtype: object
----------------------------------------
Missing values of data: 
 car_name          1
year              0
price             0
assemble_place    0
series            0
km                0
engine_type       0
transmission      0
url               0
dtype: int64
----------------------------------------
Describe data: 
                             car_name   year      price      assemble_place  \
count                          14187  14188      14188               14188   

# Chuyển đổi kiểu dữ liệu cho các cột year, price sang kiểu số thực để có thể thực hiện các phép tính toán


In [94]:
def process_year_colum(year):
    if year.isdigit():
        return float(year)
    else:
        return 0
df_bb['year'] = df_bb['year'].astype(str).apply(process_year_colum)

In [95]:
def process_price(price):
    try:
        if price.find('Tỷ') != -1:
            ty= price.split('Tỷ')[0]
            trieu = price.split('Tỷ')[1]
            trieu = trieu.split('Triệu')[0]
            return float(ty)*1000000000 + float(trieu)*1000000
        elif price.find('Triệu') != -1:
            trieu = price.split('Triệu')[0]
            trieu = trieu.replace(' ','')
            return float(trieu)*1000000
        else:
            return 0
    except:
        return 0
df_bb['price'] = df_bb['price'].astype(str).apply(process_price)

In [96]:
# Remove 'Km' and commas from the 'km' column
df_bb['km'] = df_bb['km'].str.replace('Km', '').str.replace(',', '')

# Convert the 'km' column to numeric
df_bb['km'] = pd.to_numeric(df_bb['km'])


In [97]:
#drop all rows with driven kms < 1000, nan values in all columns
df_bb.drop(df_bb[df_bb['km'] < 1000].index, inplace=True)
df_bb.dropna(inplace=True)
df_bb.isna().sum()

car_name          0
year              0
price             0
assemble_place    0
series            0
km                0
engine_type       0
transmission      0
url               0
dtype: int64

In [98]:
#loại bỏ các xe có giá trị price = 0
df_bb.drop(df_bb[df_bb['price'] == 0].index, inplace=True)
df_bb['price'].describe()

count    1.219500e+04
mean     1.160486e+09
std      1.639408e+09
min      2.600000e+07
25%      4.700000e+08
50%      6.660000e+08
75%      1.096500e+09
max      4.380000e+10
Name: price, dtype: float64

In [99]:
df_bb['brand'] = df_bb['car_name'].str.split().str[0]
df_bb['model'] = df_bb['car_name'].str.split().str[1:].str.join(' ')

Cột brand có một số hãng có số lượng xe rất ít hoặc bị thiếu, ta sẽ gộp các hãng xe này thành một nhóm là 'other'
Nếu số xe của hãng đó nhỏ hơn 10 thì sẽ gộp vào nhóm 'Other'

In [100]:
df_bb['brand'].value_counts()
threshold = 10
car_count = df_bb['brand'].value_counts()
small_brands = car_count[car_count < threshold].index.tolist()
df_bb['brand'] = df_bb['brand'].apply(lambda x: 'Other' if x in small_brands else x)
df_bb['brand'].fillna('Other', inplace=True)

Cột engine_type: ta thấy có một vài giá trị bị thiếu, ta sẽ điền các giá trị này bằng giá trị xuất hiện nhiều nhất trong cột engine_type

In [101]:
df_bb['engine_type'] = df_bb['engine_type'].apply(lambda x: df_bb['engine_type'].value_counts().index[0] if x == '-' else x)
df_bb['engine_type'].fillna(df_bb['engine_type'].value_counts().index[0], inplace=True)

In [102]:
df_bb['transmission'] = df_bb['transmission'].apply(lambda x: df_bb['transmission'].value_counts().index[0] if x == '-' else x)
df_bb['transmission'].fillna(df_bb['transmission'].value_counts().index[0], inplace=True)

In [103]:
df_bb['engine_type'] = df_bb['engine_type'].str.split().str[0]#%% md


In [104]:
df_bb.head()

,car_name,year,price,assemble_place,series,km,engine_type,transmission,url,brand,model
0,Kia Sorento Signature 2.2 AT AWD,2020.0,8.990000e+08,Lắp ráp trong nước,SUV,60000,Dầu,Số tự động,https://bonbanh.com/xe-kia-sorento-signature-2...,Kia,Sorento Signature 2.2 AT AWD
2,Kia Sorento GATH,2018.0,5.680000e+08,Lắp ráp trong nước,SUV,80000,Xăng,Số tự động,https://bonbanh.com/xe-kia-sorento-gath-2018-5...,Kia,Sorento GATH
4,Mazda CX8 Premium AWD,2022.0,9.290000e+08,Lắp ráp trong nước,SUV,50000,Xăng,Số tự động,https://bonbanh.com/xe-mazda-cx8-premium-awd-2...,Mazda,CX8 Premium AWD
6,Nissan Almera 1.0 MT,2022.0,3.690000e+08,Nhập khẩu,Sedan,25000,Xăng,Số tay,https://bonbanh.com/xe-nissan-almera-1.0-mt-20...,Nissan,Almera 1.0 MT
7,Bentley Mulsanne 6.75 V8,2011.0,4.900000e+09,Nhập khẩu,Sedan,80000,Xăng,Số tự động,https://bonbanh.com/xe-bentley-mulsanne-6.75-v...,Bentley,Mulsanne 6.75 V8


## Clean data for oto.com.vn

In [105]:
df_oto.describe()

,year
count,341.000000
mean,2018.621701
std,3.981649
min,2004.000000
25%,2017.000000
50%,2020.000000
75%,2022.000000
max,2024.000000


In [106]:
#remove all duplicate space in 
def process_price_oto(price):
    try:
        if price.find('tỉ') != -1:
            ty= price.split('tỉ')[0]
            trieu = price.split('tỉ')[1]
            trieu = trieu.split('triệu')[0]
            return float(ty)*1000000000 + float(trieu)*1000000
        elif price.find('triệu') != -1:
            trieu = price.split('triệu')[0]
            trieu = trieu.replace(' ','')
            return float(trieu)*1000000
        else:
            return 0
    except:
        return 0
df_oto['price'] = df_oto['price'].astype(str).apply(process_price_oto)

In [107]:
df_oto['price'].describe()

count    3.410000e+02
mean     1.248147e+09
std      2.018389e+09
min      1.280000e+08
25%      4.890000e+08
50%      6.680000e+08
75%      1.110000e+09
max      2.299900e+10
Name: price, dtype: float64

In [108]:
df_oto['year'].describe()

count     341.000000
mean     2018.621701
std         3.981649
min      2004.000000
25%      2017.000000
50%      2020.000000
75%      2022.000000
max      2024.000000
Name: year, dtype: float64

# Remove 'Km' and commas from the 'km' column

In [109]:
df_oto['km'] = df_oto['km'].str.replace('km', '').str.replace('.', '')

In [110]:
df_oto['km'] = pd.to_numeric(df_oto['km'])

In [111]:
df_oto['km'].describe()

count       341.000000
mean      49082.058651
std       33983.675113
min           1.000000
25%       22000.000000
50%       45000.000000
75%       70000.000000
max      188000.000000
Name: km, dtype: float64

In [112]:
df_oto['brand'] = df_oto['car_name'].str.split().str[0]
df_oto['model'] = df_oto['car_name'].str.split().str[1:].str.join(' ')

In [113]:
df_oto['brand'].value_counts()

brand
Toyota           55
Ford             45
Hyundai          44
Mercedes-Benz    42
Mazda            34
Kia              31
Lexus            12
Mitsubishi       11
Porsche           7
BMW               7
Honda             6
Chevrolet         6
Audi              6
Land              5
MG                5
VinFast           5
Bentley           3
Nissan            3
Peugeot           3
Acura             2
Mini              1
Volvo             1
Cadillac          1
Ferrari           1
Isuzu             1
Rolls-Royce       1
Suzuki            1
Daewoo            1
Volkswagen        1
Name: count, dtype: int64

In [114]:
df_oto['model'].value_counts()

model
3 Sedan 1.5L Luxury       8
Ranger Wildtrak 4x4 AT    8
Grand i10                 7
Xpander 1.5 AT            6
CR-V 1.5 L                4
                         ..
RX 350 Premium            1
520i Luxury Line          1
Innova 2.0G               1
Morning Van 1.0 AT        1
Sorento 2.4AT             1
Name: count, Length: 225, dtype: int64

In [115]:
#convert máy xăng -> xăng( lấy từ thứ 2 và viết hoa chữ đầu)
df_oto['engine_type'] = df_oto['engine_type'].str.split().str[1].str.capitalize()

In [116]:
df_oto['engine_type'].value_counts()

engine_type
Xăng    297
Dầu      42
Name: count, dtype: int64

In [117]:
# nếu là số sàn thì chuyển thành số tay, số hỗn hợp có ít nên chuyển thành số tự động

df_oto['transmission'] = df_oto['transmission'].apply(lambda x: 'Số tay' if x == 'Số sàn' else 'Số tự động')

In [118]:
df_oto['transmission'].value_counts()

transmission
Số tự động    308
Số tay         33
Name: count, dtype: int64

In [119]:
df_oto['assemble_place'] = df_oto['assemble_place'].apply(lambda x: 'Lắp ráp trong nước' if x == 'Trong nước' else 'Nhập khẩu')

In [120]:
df_oto.head()

,car_name,year,price,assemble_place,series,km,engine_type,transmission,url,brand,model
0,Kia Carnival 2.2D Signature,2022,1.360000e+09,Lắp ráp trong nước,MPV,70000,Dầu,Số tự động,https://oto.com.vn//mua-ban-xe-kia-carnival-ha...,Kia,Carnival 2.2D Signature
1,Honda CR-V 1.5 L,2023,9.850000e+08,Lắp ráp trong nước,SUV,7000,Xăng,Số tự động,https://oto.com.vn//mua-ban-xe-honda-cr-v-hcm/...,Honda,CR-V 1.5 L
2,Mazda 3 1.5L Luxury,2018,4.750000e+08,Lắp ráp trong nước,Sedan,40000,Xăng,Số tự động,https://oto.com.vn//mua-ban-xe-mazda-3-hcm/odo...,Mazda,3 1.5L Luxury
3,Mazda 3 Sedan 1.5L Luxury,2022,6.200000e+08,Lắp ráp trong nước,Sedan,7000,Xăng,Số tự động,https://oto.com.vn//mua-ban-xe-mazda-3-ha-noi/...,Mazda,3 Sedan 1.5L Luxury
4,Chevrolet Colorado LT 2.5 MT 4x2,2017,3.850000e+08,Lắp ráp trong nước,Bán Tải,90000,Xăng,Số tay,https://oto.com.vn//mua-ban-xe-chevrolet-color...,Chevrolet,Colorado LT 2.5 MT 4x2


In [121]:
df_bb.head()

,car_name,year,price,assemble_place,series,km,engine_type,transmission,url,brand,model
0,Kia Sorento Signature 2.2 AT AWD,2020.0,8.990000e+08,Lắp ráp trong nước,SUV,60000,Dầu,Số tự động,https://bonbanh.com/xe-kia-sorento-signature-2...,Kia,Sorento Signature 2.2 AT AWD
2,Kia Sorento GATH,2018.0,5.680000e+08,Lắp ráp trong nước,SUV,80000,Xăng,Số tự động,https://bonbanh.com/xe-kia-sorento-gath-2018-5...,Kia,Sorento GATH
4,Mazda CX8 Premium AWD,2022.0,9.290000e+08,Lắp ráp trong nước,SUV,50000,Xăng,Số tự động,https://bonbanh.com/xe-mazda-cx8-premium-awd-2...,Mazda,CX8 Premium AWD
6,Nissan Almera 1.0 MT,2022.0,3.690000e+08,Nhập khẩu,Sedan,25000,Xăng,Số tay,https://bonbanh.com/xe-nissan-almera-1.0-mt-20...,Nissan,Almera 1.0 MT
7,Bentley Mulsanne 6.75 V8,2011.0,4.900000e+09,Nhập khẩu,Sedan,80000,Xăng,Số tự động,https://bonbanh.com/xe-bentley-mulsanne-6.75-v...,Bentley,Mulsanne 6.75 V8


# Kết hợp 2 bảng dữ liệu

In [122]:
df = pd.concat([df_bb, df_oto], ignore_index=True)
df.head()

,car_name,year,price,assemble_place,series,km,engine_type,transmission,url,brand,model
0,Kia Sorento Signature 2.2 AT AWD,2020.0,8.990000e+08,Lắp ráp trong nước,SUV,60000,Dầu,Số tự động,https://bonbanh.com/xe-kia-sorento-signature-2...,Kia,Sorento Signature 2.2 AT AWD
1,Kia Sorento GATH,2018.0,5.680000e+08,Lắp ráp trong nước,SUV,80000,Xăng,Số tự động,https://bonbanh.com/xe-kia-sorento-gath-2018-5...,Kia,Sorento GATH
2,Mazda CX8 Premium AWD,2022.0,9.290000e+08,Lắp ráp trong nước,SUV,50000,Xăng,Số tự động,https://bonbanh.com/xe-mazda-cx8-premium-awd-2...,Mazda,CX8 Premium AWD
3,Nissan Almera 1.0 MT,2022.0,3.690000e+08,Nhập khẩu,Sedan,25000,Xăng,Số tay,https://bonbanh.com/xe-nissan-almera-1.0-mt-20...,Nissan,Almera 1.0 MT
4,Bentley Mulsanne 6.75 V8,2011.0,4.900000e+09,Nhập khẩu,Sedan,80000,Xăng,Số tự động,https://bonbanh.com/xe-bentley-mulsanne-6.75-v...,Bentley,Mulsanne 6.75 V8


In [123]:
# thay these các giá trị null trong series bằng giá trị xuất hiện nhiều nhất
def fill_null(series):
    return series.fillna(series.value_counts().index[0])
df['series'] = fill_null(df['series'])

In [124]:
df['engine_type'] = fill_null(df['engine_type'])

In [125]:
# Create a dictionary that maps the old series names to the new ones
series_dict = {
    'SUV': 'SUV',
    'Sedan': 'Sedan',
    'Crossover': 'Crossover',
    'Hatchback': 'Hatchback',
    'Bán tải / Pickup': 'Pickup',
    'Van/Minivan': 'Van/Minivan',
    'Coupe': 'Coupe',
    'Convertible/Cabriolet': 'Convertible',
    'Truck': 'Truck',
    'MPV': 'MPV',
    'Bán Tải': 'Pickup',
    'Convertible': 'Convertible'
}

# Replace the old series names with the new ones
df['series'] = df['series'].map(series_dict)


In [126]:
model_counts = df['model'].value_counts()

# Danh sách các model chỉ xuất hiện một lần
rare_models = model_counts[model_counts == 1].index

# Thay thế các model hiếm bằng 'Other'
df['model'] = df['model'].replace(rare_models, 'Other')

In [127]:
df['series'].value_counts()

series
SUV            5100
Sedan          4107
Crossover      1125
Hatchback       768
Pickup          720
Van/Minivan     530
Coupe            88
Convertible      40
Truck            29
MPV              29
Name: count, dtype: int64

In [128]:
#dump data
df.to_csv('../data/data.csv', index=False)

In [129]:
df.head()

,car_name,year,price,assemble_place,series,km,engine_type,transmission,url,brand,model
0,Kia Sorento Signature 2.2 AT AWD,2020.0,8.990000e+08,Lắp ráp trong nước,SUV,60000,Dầu,Số tự động,https://bonbanh.com/xe-kia-sorento-signature-2...,Kia,Sorento Signature 2.2 AT AWD
1,Kia Sorento GATH,2018.0,5.680000e+08,Lắp ráp trong nước,SUV,80000,Xăng,Số tự động,https://bonbanh.com/xe-kia-sorento-gath-2018-5...,Kia,Sorento GATH
2,Mazda CX8 Premium AWD,2022.0,9.290000e+08,Lắp ráp trong nước,SUV,50000,Xăng,Số tự động,https://bonbanh.com/xe-mazda-cx8-premium-awd-2...,Mazda,CX8 Premium AWD
3,Nissan Almera 1.0 MT,2022.0,3.690000e+08,Nhập khẩu,Sedan,25000,Xăng,Số tay,https://bonbanh.com/xe-nissan-almera-1.0-mt-20...,Nissan,Almera 1.0 MT
4,Bentley Mulsanne 6.75 V8,2011.0,4.900000e+09,Nhập khẩu,Sedan,80000,Xăng,Số tự động,https://bonbanh.com/xe-bentley-mulsanne-6.75-v...,Bentley,Mulsanne 6.75 V8


In [130]:
df.isnull().sum()

car_name          0
year              0
price             0
assemble_place    0
series            0
km                0
engine_type       0
transmission      0
url               0
brand             0
model             0
dtype: int64